# Downloading Transiting Planet Data

Outstanding thoughts:
- restricted to transiting planets? See if this works for tau boo.

# Learning Goals

By the end of this tutorial, you will:

- Understand how MAST makes its transiting exoplanet time-series data accessible.
- Be able to download MAST-hosted data for specific exoplanets.
- Become familiar with the exo.MAST, astoquery, and MAST APIs.
- Sort MAST data product metadata by attributes (e.g., year, PI).

# Introduction

A number of space-based missions — e.g., the Hubble Space Telescope (HST), the Transiting Exoplanet Survey Satellite (TESS), the Spitzer Space Telescope — have taken data of exoplanets and their host stars. MAST aims to make it straightforward to download datasets from these disparate sources. In this tutorial, we will cover how to explore, sort, and MAST-hosted exoplanet data using a variety of methods.


# Imports

- *ast* (Python builtin) to safely evaluate strings.
- *sys* (Python builtin) to read our system's Python version.
- *os* (Python builtin) to create directories for downloaded data.
- *json* (Python builtin) to load results from HTTP GET requests.
- *pprint* (Python builtin* to neatly print json outputs.
- *requests* (Python builtin) to make requests to MAST servers.
- *urllib.request* (Python builtin) to submit HTTP GET requests and interact with the exo.MAST API. Todo: merge with requests?
- *numpy* to help aggregate our metadata (can probably remove this dependency).
- *astropy* to provide data structures that organize our metadata.
- *astroquery.mast* to interact with MAST data products.

In [ ]:
import ast
import sys
import os
import json
import pprint
import requests
import urllib.request
from urllib.parse import quote as urlencode

import numpy as np
from astropy.table import unique, Table
from astroquery.mast import Observations

# Main Contents

## Method 1: exo.MAST

[exo.MAST](https://exo.mast.stsci.edu/docs/) is a simplified version of the full MAST API that allows for straightforward querying of exoplanet data products and metadata. Our first approach to downloading these data will use this API.

This portion of the tutorial is inspired by the exo.MAST documentation, which can be found [here](https://exo.mast.stsci.edu/docs/getting_started.html).

First, we'll use a small function that makes, receives, and parses requests to the exo.MAST api.

In [ ]:
def make_exo_mast_query(url_suffix):
    """Makes a query using the exo.MAST API. Requires internet connection.
    
     Parameters
        ----------
        url_suffix (string): the end of the URL for the HTTP GET request.
        
        Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    request_url = "https://exo.mast.stsci.edu/api/v0.1/" + url_suffix
    print(request_url)
    # make the request
    contents = urllib.request.urlopen(request_url).read()
    
    # we need to decode the returned bytes into a string
    dict_str = contents.decode("UTF-8")
    
    # null is not Pythonic
    dict_str = dict_str.replace('null', 'None')
    
    
    # File outputs can't be parsed as dictionaries directly.
    try:
        # literal_eval is safer than eval!
        contents = ast.literal_eval(dict_str)
    except:
        print('Could not parse')
        contents = dict_str
        
    return contents

Let's try this out by requesting the identifiers for a planet with multiple names: HAT-P-11 b (Bakos et al. 2010).

In [ ]:
planet_name = 'HAT-P-11 b'

# need to format the name for the query 
planet_name_formatted = planet_name.replace(' ', '%20')

request_string = f'exoplanets/identifiers/?name={planet_name_formatted}'

contents = make_exo_mast_query(request_string)


In [ ]:
contents

Let's see what files are available for this planet.

In [ ]:
# get list of spectra
request_string = f'spectra/{planet_name_formatted}/filelist/'

contents = make_exo_mast_query(request_string)

In [ ]:
contents

There are two files corresponding to spectra for this planet. Next, we can download them.

In [ ]:
filename = contents['filenames'][0]
request_string = f'spectra/{planet_name_formatted}/file/{filename}'
contents = make_exo_mast_query(request_string)

We can now write this data to a file.

In [ ]:
with open(filename, 'w') as f:
    f.write(contents)

Note, however, that not all MAST data products are accessible by this method. (why is this, exactly?)

To download other data products, we turn to other methods.

## Method 2: Using astroquery.mast.

For more MAST data products, we can make use of the astroquery.mast functionality. This approach requires an additional dependency (the [astroquery](https://astroquery.readthedocs.io/en/latest/) package).

This portion of the tutorial is inspired by the astroquery.mast tutorial, which can be found [here](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

First, let's search for all MAST data products for the hottest known exoplanet (at the time of this notebook's creation), KELT-9 b (Gaudi et al. 2017).

In [ ]:
search_radius = ".02 deg"

planet_name = 'KELT-9 b'
obs_table = Observations.query_object(planet_name,radius=search_radius)
print(obs_table[:10])  

In [ ]:
obs_table.columns

Let's sort these observations by the proposing PI (principal investigator) and filter some columns out.

In [ ]:
obs_table.sort('proposal_pi')

In [ ]:
print(obs_table[['proposal_pi', 'provenance_name', 'dataproduct_type']])

If we're interested in a specific instrument, we can next see which instruments observed this target.

In [ ]:
from astropy.table import unique

In [ ]:
print(unique(obs_table,  keys=['provenance_name'])['provenance_name'])

Great! In just a few lines, we've collected the metadata for many observations of this target into an astropy Table. Next, let's see what data products are available for the most recent QLP observation.

In [ ]:
obs_table_qlp = obs_table[obs_table['provenance_name']=='QLP']
obs_table_qlp.sort('t_min')
data_products_by_obs = Observations.get_product_list(obs_table_qlp[-1])
print(data_products_by_obs) 

There are two timeseries data products. Let's download the first one. 

In [ ]:
data_products_by_obs.columns

In [ ]:
obs_collection = data_products_by_obs['obs_collection'][0]
obs_id = data_products_by_obs['obs_id'][0]

single_obs = Observations.query_criteria(obs_collection=obs_collection, obs_id=obs_id)
data_products = Observations.get_product_list(single_obs)

manifest = Observations.download_products(data_products, productType="SCIENCE")

In [ ]:
print(manifest)

We've now successfully downloaded MAST data using the astroquery.mast API.

## Method 3: Directly using the MAST API.
The final approach is a bit more hands-on and requires more code, but it allows for the most flexibility. Additionally, it provides the most insight into what's going on "under the hood" with the MAST requests. This approach requires the [Astropy](https://www.astropy.org/) and [NumPy](https://numpy.org/) dependencies.

This portion of the tutorial is inspired by the general MAST API tutorial, which can be found [here](_https://mast.stsci.edu/api/v0/MastApiTutorial.html).

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

We'll start by defining a function that processes MAST queries.

In [ ]:
def mast_query(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    # Base API url
    request_url='https://mast.stsci.edu/api/v0/invoke'    
    
    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create HTTP Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    req_string = json.dumps(request)
    req_string = urlencode(req_string)
    
    # Perform the HTTP request
    resp = requests.post(request_url, data="request="+req_string, headers=headers)
    
    # Pull out the headers and response content
    head = resp.headers
    content = resp.content.decode('utf-8')

    return head, content

Sticking with out previous example, let's look at the planet KELT-9 b.

In [ ]:
object_of_interest = 'KELT-9 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

headers, resolved_object_string = mast_query(resolver_request)

resolved_object = json.loads(resolved_object_string)

pp.pprint(resolved_object)

TODO: complete.
Parsing apart the output:
- the *cached* field denotes whether this result has already been saved on this device.
- the *canonicalName* field denotes the default name of the planet.
- the *decl* field denotes the declination of the resolved coordinate.
- the *ra* field denotes the right ascention of the resolved coordinate.
- the *resolver* field denotes which algorithm (?) was used to determine this coordinate.
- the *resolverTime* field denotes how long the resolver took to resolve the coordinate (in ms?)
- the *searchRadius* field denotes the raidius of the search.
- the *searchString* field denotes ...

Now that we've resolved our target, let's save its coordinates as variables — we'll need them later on.

In [ ]:
obj_ra = resolved_object['resolvedCoordinate'][0]['ra']
obj_dec = resolved_object['resolvedCoordinate'][0]['decl']

With the coordinates of the object now known, we can run a *Mast.Caom.Cone* query to retrieve metadata on all MAST data around this coordinate.

In [ ]:
mast_request = {'service':'Mast.Caom.Cone',
                'params':{'ra':obj_ra,
                          'dec':obj_dec,
                          'radius':0.2},
                'format':'json',
                'pagesize':2000,
                'page':1,
                'removenullcolumns':True,
                'removecache':True}

headers, mast_data_str = mast_query(mast_request)

mast_data = json.loads(mast_data_str)

print(mast_data.keys())
print("Query status:",mast_data['status'])

Let's take a look at the first returned data entry.

In [ ]:
pp.pprint(mast_data['data'][0])

There's a lot of metadata here, and it's a bit hard to understand all at once. To make things a bit more digestible, we can create an astropy Table.

In [ ]:
mast_data_table = Table()

for col,atype in [(x['name'],x['type']) for x in mast_data['fields']]:
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    mast_data_table[col] = np.array([x.get(col,None) for x in mast_data['data']],dtype=atype)
    
print(mast_data_table)

In [ ]:
mast_data_table.sort('t_min')

With our metadata all acquired, we can now sort it based on, e.g., start date.

In [ ]:
print(mast_data_table)

Let's get most recent (HST) data product.

In [ ]:
# Picking the first Hubble Space Telescope observation
interesting_observation = mast_data_table[mast_data_table["obs_collection"] == "HST"][-1]
print("Observation:",
      [interesting_observation[x] for x in ['dataproduct_type', 'obs_collection', 'instrument_name']])

It appears that the latest HST data for this target is a STIS (what's that acronym?) spectrum.

We can now make another MAST query to determine how many data products are associated with this observation.

In [ ]:
obsid = interesting_observation['obsid']

product_request = {'service':'Mast.Caom.Products',
                  'params':{'obsid':obsid},
                  'format':'json',
                  'pagesize':100,
                  'page':1}   

headers, obs_products_string = mast_query(product_request)

obs_products = json.loads(obs_products_string)

print("Number of data products:", len(obs_products["data"]))
print("Product information column names:")
pp.pprint(obs_products['fields'])

We can also see what *types* these data products are.

In [ ]:
pp.pprint([x.get('productType',"") for x in obs_products["data"]])

We can place these results in a table as well, restricting ourselves to the science products.

In [ ]:
sci_prod_arr = [x for x in obs_products['data'] if x.get("productType", None) == 'SCIENCE']
science_products = Table()

for col, atype in [(x['name'], x['type']) for x in obs_products['fields']]:
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    if atype == "int":
        atype = "float" # array may contain nan values, and they do not exist in numpy integer arrays
    science_products[col] = np.array([x.get(col,None) for x in sci_prod_arr],dtype=atype)

print("Number of science products:",len(science_products))
print(science_products)

Next, let's download these data products.

In [ ]:
download_url = 'https://mast.stsci.edu/api/v0.1/Download/file?'

for row in science_products:     

    # make file path
    out_path = os.path.join("mastFiles", row['obs_collection'], row['obs_id'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    out_path = os.path.join(out_path, os.path.basename(row['productFilename']))
        
    # Download the data
    payload = {"uri":row['dataURI']}
    resp = requests.get(download_url, params=payload)
    
    # save to file
    with open(out_path,'wb') as FLE:
        FLE.write(resp.content)
        
    # check for file 
    if not os.path.isfile(out_path):
        print("ERROR: " + out_path + " failed to download.")
    else:
        print("COMPLETE: ", out_path)

You can check that these data files have been downloaded correctly by checking `out_path`.

In [ ]:
ls mastFiles/HST/oecfz2050/

Great! We've successfully downloaded the data for this planet via MAST.

# Exercises
- Repeat this for Spitzer?
- Query a light curve?
- Read in and plot the data? Compare to a downloaded plot?

# Additional Resources
- Here's an [introduction to HTTP GET requests](https://www.ibm.com/docs/en/cics-ts/5.3?topic=protocol-http-requests).
- Primer on exoplanet data types
- Here's a [neat blog post](https://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html) on why `literal_eval` is preferred to `eval`.
- [Prettyprint documentation](https://docs.python.org/3/library/pprint.html)

# About this Notebook

# Citations
- [Gaudi, B. Scott, et al. "A giant planet undergoing extreme-ultraviolet irradiation by its hot massive-star host." Nature 546.7659 (2017): 514-518.](https://www.nature.com/articles/nature22392)
- [Bakos, G. Á., et al. "HAT-P-11b: A super-Neptune planet transiting a bright K star in the Kepler field." The Astrophysical Journal 710.2 (2010): 1724.](https://iopscience.iop.org/article/10.1088/0004-637X/710/2/1724/meta)
- [exo.MAST tutorial](https://exo.mast.stsci.edu/docs/getting_started.html#resolving-exoplanets)
- [astroquery.MAST documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
- [MAST API documentation](https://mast.stsci.edu/api/v0/MastApiTutorial.html)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>
